In [1]:
import pandas as pd
import random
import numpy as np
import pickle
import keras
from keras.layers import *
from keras import backend as k
from keras import optimizers
from keras.models import Model
from time import time
from keras.models import load_model
from google.colab import files
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, normalized_mutual_info_score

Using TensorFlow backend.


In [0]:
uploaded = files.upload()

In [0]:
mfccs1 = np.load('mfccs1.npy')
#mfccs2 = np.load('mfccs2.npy')
#mfccs3 = np.load('mfccs3.npy')
#mfccs4 = np.load('mfccs4.npy')

In [0]:
pitches1 = np.load('pitches1.npy')
#pitches2 = np.load('pitches2.npy')
#pitches3 = np.load('pitches3.npy')
#pitches4 = np.load('pitches4.npy')

In [4]:
mfccs2.shape

(2004, 1500, 12)

In [0]:
mfccs12 = np.append(mfccs1, mfccs2, axis = 0)
mfccs34 = np.append(mfccs3, mfccs4, axis = 0)
mfccs = np.append(mfccs12, mfccs34, axis = 0)

pitches12 = np.append(pitches1, pitches2, axis = 0)
pitches34 = np.append(pitches3, pitches4, axis = 0)
pitches = np.append(pitches12, pitches34, axis = 0)

In [0]:
loudness_ = np.load('loudness.npy')

In [0]:
loudness1 = loudness_[:2004]

In [0]:
features = np.append(mfccs1, pitches1, axis = 2)
features_ = np.append(features, loudness1, axis = 2)

In [7]:
features_.shape

(2004, 1500, 25)

In [0]:
features_dim = 1500
wide = 25
BATCH_SIZE  = 32
N1 = 32
N2 = 32
Nfc = 520
L1 = 30
L2 = 15

In [0]:
batches = int(len(features_)/BATCH_SIZE)

In [0]:
def batch_generator():
    
  while True:
    
    for batch in range(batches):
      x_ = []
      
      data_ = features_[batch * BATCH_SIZE: (batch + 1) * BATCH_SIZE]
      for i in data_:
          x_.append(np.reshape(i, (features_dim, wide)))

      x = np.asarray(x_)
      
      yield (x, x)

In [0]:
#autoencoder = load_model('model1.h5')

In [0]:
#ENCODER

input1 = Input(shape = (features_dim, wide))

encoder = Conv1D(N1, L1, strides = 1, padding = 'SAME', use_bias = True, 
                 bias_initializer = keras.initializers.Constant(value=0.1), activation = 'relu')(input1)

encoder = MaxPooling1D(pool_size = 2, strides = 2)(encoder)

encoder = Conv1D(N2, L2, strides = 1, padding = 'SAME', use_bias = True, 
                 bias_initializer = keras.initializers.Constant(value=0.1), activation = 'relu')(encoder)

encoder = MaxPooling1D(pool_size = 2, strides = 2)(encoder)

out_shape = list(k.int_shape(encoder))

encoder = Flatten()(encoder)

encoder = Dense(Nfc, activation = 'relu')(encoder)

encoder = Dropout(0.5)(encoder)

encoder = Dense(20, activation = 'sigmoid')(encoder)


#DECODER

decoder = Dense((out_shape[1] * out_shape[2]), activation = 'relu')(encoder)

decoder = Dropout(0.5)(decoder)

decoder = Reshape((out_shape[1], out_shape[2]))(decoder)

decoder = UpSampling1D(size = 2)(decoder)

decoder = Conv1D(N2, L2, strides = 1, padding = 'SAME', use_bias = True, 
                 bias_initializer = keras.initializers.Constant(value=0.1), activation = 'relu')(decoder)

decoder = UpSampling1D(size = 2)(decoder)

decoder = Conv1D(N1, L1, strides = 1, padding = 'SAME', use_bias = True, 
                 bias_initializer = keras.initializers.Constant(value=0.1), activation = 'relu')(decoder)

out = Conv1D(wide, 1, padding = 'same', activation = 'sigmoid')(decoder)


autoencoder = Model(input1, out)

In [12]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1500, 25)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1500, 32)          24032     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 750, 32)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 750, 32)           15392     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 375, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 520)               6240520   
__________

In [0]:
encoder_ = Model(input1, encoder)

In [14]:
encoder_.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1500, 25)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1500, 32)          24032     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 750, 32)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 750, 32)           15392     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 375, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 520)               6240520   
__________

In [0]:
adam_ = optimizers.Adam(lr = 1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

In [0]:
autoencoder.compile(loss = 'binary_crossentropy', optimizer = adam_)

In [17]:
autoencoder.fit_generator(batch_generator(), steps_per_epoch = batches, epochs = 10, shuffle = True)

Epoch 1/10
62/62 [==============================] - 67s 1s/step - loss: -4.6409
Epoch 2/10
62/62 [==============================] - 64s 1s/step - loss: -38.0008
Epoch 3/10
62/62 [==============================] - 63s 1s/step - loss: -49.8971
Epoch 4/10
62/62 [==============================] - 63s 1s/step - loss: -53.5330
Epoch 5/10
62/62 [==============================] - 63s 1s/step - loss: -58.6253
Epoch 6/10
62/62 [==============================] - 63s 1s/step - loss: -62.8165
Epoch 7/10
62/62 [==============================] - 64s 1s/step - loss: -65.0066
Epoch 8/10
62/62 [==============================] - 63s 1s/step - loss: -66.2867
Epoch 9/10
62/62 [==============================] - 63s 1s/step - loss: -67.3125
Epoch 10/10
62/62 [==============================] - 63s 1s/step - loss: -68.1239


In [0]:
autoencoder.save('model2.h5')

In [0]:
files.download('model2.h5')

In [0]:
km = KMeans(n_jobs=-1, n_clusters=10, n_init=20)

In [0]:
encode_pred = encoder_.predict(data)